In [ ]:
# Authentication Procedure

import base64
import json
import sys
import requests
requests.packages.urllib3.disable_warnings()

# Authentication ESM parameters
host = "192.168.20.130"
user = "NGCP"
passwd = "McAfee123!"

# Login URL
url = 'https://{}/rs/esm/login'.format(host)

# Building authentication header
creds = '{}:{}'.format(user, passwd)
v9_b64_creds = base64.b64encode(creds.encode('utf-8'))

headers = {'Authorization': 'Basic ' + v9_b64_creds.decode('utf-8'),
           'Content-Type': 'application/json'}

# Building data parameters
v10_b64_user   = base64.b64encode(user.encode('utf-8')).decode()
v10_b64_passwd = base64.b64encode(passwd.encode('utf-8')).decode()

data = {"username": v10_b64_user, 
        "password": v10_b64_passwd,
        "locale": "en_US",
        "os":"Win32"}



# Connect to the server RESTful API
try:
    r = requests.post(url, data=json.dumps(data), headers=headers, verify=False)
    
except requests.exceptions.ConnectionError:
    print 'Error connecting to ESM'
    sys.exit(1)

# Minimum  error control
if r.status_code in [400, 401]:
    print 'Ivalid Username or password'
    sys.exit(1)
elif 402 <= r.status_code <= 600:
    print'ESM Login Error: ', r.text
    sys.exit(1)
    
print 'Connection status code: %s '%(r.status_code)

# With the information obtained we build the authenticated header 
# This auth_headerwill be used in further calls to the API
auth_header                 = {'Content-Type': 'application/json'}
auth_header['Cookie']       = r.headers.get('Set-Cookie')
auth_header['X-Xsrf-Token'] = r.headers.get('Xsrf-Token')
auth_header['SID']          = r.headers.get('Location')

print auth_header

In [25]:
# ESM time procedure

# Time URL
url = 'https://{}/rs/esm/essmgtGetESSTime'.format(host)
data = None

r = requests.post(url, data = data, headers = auth_header, verify = False)

print 'Connection status code: %s '%(r.status_code)

r = r.json()['return']

print 'SIEM Time: %s'%r['value']

Connection status code: 200 
SIEM Time: 2018-05-25T10:18:11.0+0000


In [26]:
# Getting builtstamp from ESM

# Build Stamp URL
url = 'https://{}/rs/esm/essmgtGetBuildStamp'.format(host)
data = None

r = requests.post(url, data = data, headers = auth_header, verify = False)

print 'Connection status code: %s '%(r.status_code)

r = r.json()['return']

print 'SIEM buildStamp: %s'%r['buildStamp']
siem_ver = r['buildStamp'].split()[0]
print 'SIEM version %s'%siem_ver

Connection status code: 200 
SIEM buildStamp: 10.2.0 20171116224008
SIEM version 10.2.0


In [27]:
# Getting the status of the system

# Status URL
url = 'https://{}/rs/esm/sysGetSysInfo'.format(host)
data = None

r = requests.post(url, data = data, headers = auth_header, verify = False)

print 'Connection status code: %s '%(r.status_code)

r = r.json()['return']

print 'SIEM General status: %s'%r
siem_disk_info = r['hdd']
print '\n\nSIEM Disk information: \t%s'%siem_disk_info
siem_ram_info = r['ram']
print "SIEM Ram information: \t%s"%siem_ram_info

Connection status code: 200 
SIEM General status: {u'autoBackupEnabled': True, u'statusAndAlertsCheckHour': 0, u'ram': u'Avail: 7978MB, Used: 7415MB, Free: 562MB', u'rulesAndSoftNextCheck': u'2018/03/07 07:57:00', u'hdd': u'sda3     Size:  241GB, Used:   11GB( 5%), Available:  218GB, Mount: /', u'rulesAndSoftLastCheck': u'2018/03/06 19:57:10', u'overviewItems': [{u'log': 1, u'flow': 1, u'alert': 1, u'hasRights': True, u'version': 10002000, u'model': u'ENMELM-VM8', u'label': u'Local Receiver-ELM', u'ipsid': 144116287587483648L}, {u'log': 1, u'flow': 0, u'alert': 1, u'hasRights': True, u'version': 10002000, u'model': u'ELS-VM8', u'label': u'ELS Engine', u'ipsid': 144118486610739200L}, {u'log': 1, u'flow': 1, u'alert': 1, u'hasRights': True, u'version': 10002000, u'model': u'ACE-VM8', u'label': u'ACE Engine', u'ipsid': 144119586122366976L}], u'backupLastTime': u'2018/05/18 13:03:03', u'backupNextTime': u'2018/05/25 13:03', u'rulesAndSoftwareExpireTime': u'', u'statusAndAlertNextCheckIn': 

In [28]:
# Procedure to query for events

'''
Description: This procedure execute a query against the ESM to get information from

    Input:       fields       --> List of fields we want to get from
                 filters      --> Dic with the following format: {'field':'field_value', 
                                                                  'operator': 'type_of_operator', 
                                                                  'values':'value' }
                 time_range   --> Valid Period of time to get the information from
                 custom_start --> if time range is CUSTOM, custom_start identify the start time
                 custom_end   --> if time_range is CUSTOM, custo_end identify the finish time
                 limit        --> Number of rows (logs entries) we want to get, 0 equal unlimitted.
                 total        --> If we want to get the total values, by defualt 0

    Output:		 Job_ID, this value will be used later on to check the status of the query 
                and get the final information

    Example Json:

            {"config": {
                        "timeRange": "CUSTOM",
                        "customStart": "2014-12-19T12:31:02.203Z",
                        "customEnd": "2014-12-19T12:31:02.204Z",
                        "order": 
                            [{
                            "direction": "ASCENDING",
                            "field": {"name": "(name)"}
                            }],
                        "includeTotal": false,
                        "fields": [{"name": "(name)"}],
                        "filters": 
                            [{
                            "type": "EsmFieldFilter",
                            "field": {"name": "(name)"},
                            "operator": "IN",
                            "values": 
                                [{
                                "type": "EsmBasicValue",
                                "value": "(value)"
                                }]
                            }],
                        "limit": 0
                    }
            }
'''
fields=None 
filters=None
time_range='LAST_HOUR'
custom_start = None
custom_end = None
limit=0
total=False

#--> We are filtering using the value 57 = Correlated Engine and 380 = ACE CORRELATION
filters = {'field':'DSID', 'operator': 'IN', 'values':'47, 380'} 
#filters = {'field':'IPSID', 'operator': 'IN', 'values': ipsid_value} 
#filters = {'field':'ID', 'operator': 'IN', 'values': '1119663'} #--> Ejemplo de fitlro por identificador de evento

fields  = ['FirstTime','LastTime','ThirdPartyType.Name','DSID','IPSID','AlertID',\
          'Rule_NDSNormSigID.msg','Rule.msg','Rule_Name','Action.Name','EventCount','Severity','AvgSeverity',\
          'SrcIP','SrcPort','DstIP','DstPort','Protocol',\
          'UserIDSrc','UserIDDst','Message_Text','ID','GeoLoc_ASNGeoDst.Msg','GeoLoc_ASNGeoSrc.Msg']


time_range = 'CURRENT_YEAR'


url = 'https://{}/rs/esm/qryExecuteDetail'.format(host)

list_of_fields  = []
list_of_filters = []

if fields:
    for each_field in fields:
        list_of_fields.append({'name': each_field})
else:
    list_of_fields = []


if filters:
    list_of_filters.append({"type": "EsmFieldFilter","field": {"name": filters['field']}, 
                            "operator": filters['operator'], 
                            "values":[{"type": "EsmBasicValue", 
                                       "value": filters['values']
                                      }]})
else:
    list_of_filters = []

# BETA #

order = [{
         "direction": "DESCENDING", 
         "field": {"name": "LastTime"}
        }]


if time_range == 'CUSTOM':

    payload =  {'config' : 
                    {
                    'timeRange'    : time_range,
                    'customStart'  : custom_start,
                    'customEnd'    : custom_end,
                    'order'        : order,
                    'includeTotal' : total,
                    'fields'       : list_of_fields,
                    'filters'      : list_of_filters,
                    'limit'        : limit
                    }
                }
else:
    payload =  {'config' : 
                    {
                    'timeRange'    : time_range,
                    'order'        : order,
                    'includeTotal' : total,
                    'fields'       : list_of_fields,
                    'filters'      : list_of_filters,
                    'limit'        : limit
                    }
                }

r = requests.post(url + '/?type=EVENT&reverse=false', data = json.dumps(payload), headers=auth_header, verify=False)

print r.status_code
r = r.json()['return']

job_id = r['resultID']['value']

print job_id
                           

200
140471397892928


In [29]:
# Getting the Status of the query

payload = {'resultID':{'value': job_id}}


url = 'https://{}/rs/esm/qryGetStatus'.format(host)

r = requests.post(url, data = json.dumps(payload), headers=auth_header, verify=False)

print r.status_code

r = r.json()

print r


200
{u'return': {u'totalRecords': 17, u'complete': True, u'milliseconds': 158, u'percentComplete': 100}}


In [20]:
# Procedure to get the list of events related to previous job_id

payload = {'resultID':{'value': job_id}}

url = 'https://{}/rs/esm/qryGetResults'.format(host)

r = requests.post(url + '/?startPos=0&numRows=5000&reverse=false', data = json.dumps(payload), 
                  headers=auth_header, verify=False)

print r.status_code

r = r.json()

print r

200
{u'return': {u'rows': [{u'values': [u'2018/03/07 23:28:11', u'2018/03/07 23:28:11', u'Correlation Engine', u'47', u'144119586139144192', u'93', u'Login', u'Suspicious - Internal IP Login without Logout', u'', u'success', u'1', u'8', u'8', u'192.168.20.1', u'0', u'192.168.20.130', u'0', u'n/a', u'NGCP', u'', u'', u'201867463052', u'', u'']}, {u'values': [u'2018/03/06 19:33:25', u'2018/03/06 19:33:25', u'Correlation Engine', u'47', u'144116287604260864', u'8494', u'Login', u'Custom - Double Authentication', u'', u'informational', u'1', u'50', u'50', u'192.168.20.210', u'50800', u'192.168.20.130', u'0', u'n/a', u'root', u'', u'Mar  6 17:33:25 McAfee sshd[1730]: Accepted password for root from 192.168.20.210 port 50800 ssh2', u'201869462914', u'', u'']}, {u'values': [u'2018/03/06 19:33:03', u'2018/03/06 19:33:03', u'Correlation Engine', u'47', u'144116287604260864', u'8493', u'Login', u'Custom - Double Authentication', u'', u'informational', u'1', u'50', u'50', u'192.168.20.1', u'58164

In [30]:
# AS my previous query is filtered by the Correlation engine, now I want all the source events associated.
IPSID   = '144116287604260864'
AlertID = '8494'

IPSID ='144119586139144192'
AlertID ='93'


correlated_event_ID = IPSID + '|' + AlertID 


fields  = ['FirstTime','LastTime','ThirdPartyType.Name','DSID','IPSID',\
          'Rule_NDSNormSigID.msg','Rule.msg','Rule_Name','Action.Name','EventCount','Severity','AvgSeverity',\
          'SrcIP','SrcPort','DstIP','DstPort','Protocol',\
          'UserIDSrc','UserIDDst','Message_Text','ID','GeoLoc_ASNGeoDst.Msg','GeoLoc_ASNGeoSrc.Msg']

list_of_fields  = []

if fields:
    for each_field in fields:
        list_of_fields.append({'name': each_field})
else:
    list_of_fields = []


payload = {'eventId': {'value':correlated_event_ID},'fields' : list_of_fields}

url = 'https://{}/rs/esm/qryGetCorrEventDataForID'.format(host)

r = requests.post(url + '?queryType=EVENT', data = json.dumps(payload), headers=auth_header, verify=False)

print r.status_code

r = r.json()

print r


200
{u'return': [{u'values': [u'2018/03/07 23:28:11', u'2018/03/07 23:28:11', u'McAfee Enterprise Security Manager', u'306', u'144115188075855872', u'Login', u'User Logon', u'', u'success', u'1', u'19', u'19', u'192.168.20.1', u'0', u'192.168.20.130', u'0', u'n/a', u'NGCP', u'', u'', u'1050', u'', u'']}]}


In [32]:
# Logout procedure

# Logout URL
url = 'https://{}/rs/esm/logout'.format(host)

r = requests.delete(url, headers=auth_header, verify=False)

print 'Connection status code: %s '%(r.status_code)

Connection status code: 401 
